In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso # Linear Regression with regularization
from sklearn.linear_model import Ridge # regularised linear regression
from sklearn.metrics import mean_squared_error

import pickle

In [2]:
# !pip install pyarrow

In [7]:
def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60) ]
    
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    return df

In [8]:
df_train = read_data('../data/green_tripdata_2021-01.parquet')
df_val = read_data('../data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
categorical = ['PU_DO']#,'PULocationID','DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
def train_model(lr = LinearRegression()):
    # For Rapid and clean experimentaion
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_train)
    print('train mse',mean_squared_error(y_train, y_pred, squared=False))
    
    y_pred = lr.predict(X_val)
    print('val mse',mean_squared_error(y_val, y_pred, squared=False))

In [12]:
train_model(LinearRegression())

train mse 4.640859708885847
val mse 7.479343826040432


In [15]:
train_model(Lasso(alpha = 0.001))

train mse 8.345478908061276
val mse 9.233436225720547


In [14]:
train_model(Ridge())

train mse 10.712682923244786
val mse 11.3426039432658


**Saving the Best Model**

In [19]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [20]:
with open('../models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv,lr), f_out)